#Load dataset

In [3]:
%pwd

'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot\\research'

In [ ]:
import os
os.chdir('../')


In [6]:
%pwd   # -- check OK :   'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot'


'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot'

In [ ]:

from langchain.document_loaders import DirectoryLoader , UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter    


def load_word_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.docx",
        loader_cls=UnstructuredWordDocumentLoader #loại file cần load   
    )
    documents = loader.load()
    return documents


In [ ]:
extracted_data = load_word_files(data='DataChatbot/')